# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory
Data_URL= 'https://github.com/jfcarmonag/nd00333-capstone/raw/master/starter_file/BankChurners.csv'
ds=TabularDatasetFactory.from_delimited_files(path=Data_URL)

ws = Workspace.from_config()
experiment_name = 'exp-capstone-hyperdrive'

experiment=Experiment(ws, experiment_name)

cluster_name = 'project-cluster'
try:
    compute_target = ComputeTarget(ws, cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [17]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal




# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=3,slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--max_depth': choice(3,4,5),
                 '--learning_rate':uniform(0.05,0.1),
                 '--gamma': choice(0.0,0.25,0.5,1.0),
                 '--reg_lambda':choice(0,1.0,10.0),
                 '--scale_pos_weight': choice (1,3,5)})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
    source_directory="./",
    entry_script="train.py",
    compute_target=compute_target,
)

hyperdrive_run_config = HyperDriveConfig(
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=50,
    max_concurrent_runs=8,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    estimator=estimator
)

In [21]:
#TODO: Submit your experiment
from azureml.widgets import RunDetails
hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_814ce2a5-ac00-4bcf-922c-abdbaf641fc8
Web View: https://ml.azure.com/experiments/exp-capstone-hyperdrive/runs/HD_814ce2a5-ac00-4bcf-922c-abdbaf641fc8?wsid=/subscriptions/94e14ad4-bf97-47e8-aae0-f9b85a7befa8/resourcegroups/aml-quickstarts-128506/workspaces/quick-starts-ws-128506

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-30T20:55:41.147318][API][INFO]Experiment created<END>\n"<START>[2020-11-30T20:55:42.8143978Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
# register model 
model = best_run.register_model(model_name='model', model_path='outputs/model.pkl')
print(f"Model {model.name}.v{model.version} correctly saved")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service